# Лабараторная 2

### Русский датасет

In [1]:
from deeppavlov import build_model
import pandas as pd
import re
import pymorphy2

In [2]:
data = pd.read_csv('./rusentitweet_full.csv', 
    engine='python', 
    on_bad_lines='skip',
)
data.head(10)

,Unnamed: 0,text,label,id
0,0,@varlamov @McFaul На,skip,1327934765807308801
1,1,велл они всё равно что мусор так что ничего с...,negative,1252943181387350017
2,2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",negative,1323610669061677056
3,3,Ой какие неожиданные результаты 🤭 https://t.co...,neutral,1336231661160247297
4,4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,neutral,1292421736454127617
5,5,@idkwhht мы тоже мебельная компания уджина😳😳😳,neutral,1303753911174606848
6,6,Счастья здоровья 10 классникам https://t.co/M9...,speech,1339917376427601921
7,7,@dntbliev НЕ ПАЛИ.,neutral,1267898207176142848
8,8,@BTS_twt ты такой красивый 😭😭😭🥺💓,positive,1281551551706865665
9,9,"@Ladyzchensk Цыган , хуле ...",negative,1257761690122358784


### Предобработка

Удаляем лишние столбцы

In [3]:
data = data.drop('Unnamed: 0', axis=1)
data = data.drop('id', axis=1)

Меняем значения и тип.

In [4]:
rep = {
    'skip': '0',
    'neutral': '0',
    'speech': '0',
    'positive': '0',
    'negative': '1'
}
data['label'] = data['label'].replace(rep)
data['label'] = data['label'].astype(int)

Векторизация в BERT включена, поэтому мы не будем тут дополнительно прописывать (как с англисйким датасетом).

Лемматизация

In [5]:
lem = pymorphy2.MorphAnalyzer()
def cls(series: pd.Series) -> pd.Series:
    return series.apply(cl)
def cl(text: str) -> str:
    return clear(lemm(text))
def clear(text: str) -> str:
    return re.sub(r'[^\w\s]', ' ', text)
def lemm(text: str) -> str:
    return ' '.join(lem.parse(word)[0].normal_form for word in text.lower().split())

In [6]:
data['newtext'] = cls(data['text'])

Смотрим результат предобработки:

In [7]:
data.head(10)

,text,label,newtext
0,@varlamov @McFaul На,0,varlamov mcfaul на
1,велл они всё равно что мусор так что ничего с...,1,велла они всё равно что мусор так что ничего с...
2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",1,трезвый жизнь какой то такой стрёмная с ар...
3,Ой какие неожиданные результаты 🤭 https://t.co...,0,ой какой неожиданный результат https t co ...
4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,0,shvonder_chief dimsmirnov175 на забор тоже н...
5,@idkwhht мы тоже мебельная компания уджина😳😳😳,0,idkwhht мы тоже мебельный компания уджина
6,Счастья здоровья 10 классникам https://t.co/M9...,0,счастие здоровье 10 классник https t co m9vu...
7,@dntbliev НЕ ПАЛИ.,0,dntbliev не пали
8,@BTS_twt ты такой красивый 😭😭😭🥺💓,0,bts_twt ты такой красивый
9,"@Ladyzchensk Цыган , хуле ...",1,ladyzchensk цыган хула


Используем модель rusentiment_convers_bert для предсказания тональности для русского текста

In [8]:
model = build_model('rusentiment_convers_bert', download=True, install=True)

2025-03-02 15:59:59.549 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/classifiers/rusentiment_convers_bert/rusentiment_convers_bert_torch.tar.gz download because of matching hashes
D:\gadd\envs\Brp2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\gadd\envs\Brp2\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias

Тут такая же проблема(как и с английским датасетом) - большой объем датасета.

Поэтому копируем в новую таблицу первую 1000 строк и работаем с ними.

In [9]:
data2 = data.head(1000).copy()
data2['res'] = model(data2['newtext'].tolist())
data.loc[:999, 'res'] = data2['res']

Итог работы:

In [10]:
data2.head(10)

,text,label,newtext,res
0,@varlamov @McFaul На,0,varlamov mcfaul на,neutral
1,велл они всё равно что мусор так что ничего с...,1,велла они всё равно что мусор так что ничего с...,negative
2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",1,трезвый жизнь какой то такой стрёмная с ар...,negative
3,Ой какие неожиданные результаты 🤭 https://t.co...,0,ой какой неожиданный результат https t co ...,positive
4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,0,shvonder_chief dimsmirnov175 на забор тоже н...,neutral
5,@idkwhht мы тоже мебельная компания уджина😳😳😳,0,idkwhht мы тоже мебельный компания уджина,neutral
6,Счастья здоровья 10 классникам https://t.co/M9...,0,счастие здоровье 10 классник https t co m9vu...,speech
7,@dntbliev НЕ ПАЛИ.,0,dntbliev не пали,neutral
8,@BTS_twt ты такой красивый 😭😭😭🥺💓,0,bts_twt ты такой красивый,positive
9,"@Ladyzchensk Цыган , хуле ...",1,ladyzchensk цыган хула,skip
